In [1]:

%matplotlib inline

from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray
import xbout
import scipy
import re

onedrive_path = onedrive_path = str(os.getcwd()).split("OneDrive")[0] + "OneDrive"
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\sdtools"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\soledge"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages"))


from gridtools.b2_tools import *
from gridtools.utils import *

from hermes3.fluxes import *
from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from code_comparison.code_comparison import *


%load_ext autoreload
%autoreload 2
%matplotlib tk

print("Done")


st40_phys_viewer can not be loaded from outside Tokamak Energy!

Done


In [2]:

import h5py
import os
import numpy											as np
import matplotlib
import matplotlib.pyplot							as pyp
from matplotlib.backends.backend_pdf	import PdfPages
from eirene.get_wall_triangle					import get_wall_triangle
from routines.h5_routines							import h5_read
from files.load_refpar_file							import load_refpar_file
from files.load_ions_list								import load_ions_list
from files.load_soledge_mesh_file				import load_soledge_mesh_file
from files.load_eirene_triangles					import load_eirene_triangles

In [12]:
path=os.path.join(onedrive_path, r"Project\collab\tech\compare_data\soledge_cases\copy_wallStd_bra_nocore_D")
rz0_line = [0.,0.]
theta_line=0.
log_scale=0
one_plot=0
exp = []
evolution = 0
save="none"

In [14]:
print("plot1d_wall_ntmpi")

Eirene = load_eirene_triangles(os.path.join(path, "triangles.h5"))

i_plot_file = 0

exp_data_ok = False
if((len(exp) > 0) and (shot > 0) and (time > 0)):
    exp_data = load_exp_data(exp, shot, time)
    if(exp_data.lp.ok): exp_data_ok = True

#	Find wall intersection along line

ZeroTriangle, ZeroSide, Ri, Zi, RWallTriangles, ZWallTriangles, iWallTriangles, iWallSide, iWallKnots  = \
            get_wall_triangle(Eirene, rz0_line=rz0_line, theta_line=theta_line, no_plot=1, no_print=1, no_triangles=0)

RWallTriangles = np.append(RWallTriangles, RWallTriangles[0])
ZWallTriangles = np.append(ZWallTriangles, ZWallTriangles[0])
iWallKnots	   = np.append(iWallKnots, iWallKnots[0])

WalldL  	   = np.sqrt((RWallTriangles[1:]-RWallTriangles[:-1])**2 + (ZWallTriangles[1:]-ZWallTriangles[:-1])**2)
DistKnots	   = np.cumsum(np.append(0., WalldL))
DistTriangles  = 0.5*(DistKnots[:-1]+DistKnots[1:])


#	Read references data

RefPar = load_refpar_file(os.path.join(path, "Results"))

if(evolution == 0):
    base_plasma_name = os.path.join(path, "Results/")
else:
    base_plasma_name = os.path.join(path, "/Evolution/{:d}_".format(evolution))

if_plasma	= h5py.File(os.path.join(base_plasma_name, "plasma_0"), "r")
Te			= h5_read(if_plasma,"triangles/temperature")*RefPar.T0eV
if_plasma.close()

if_plasma	= h5py.File(os.path.join(base_plasma_name, "plasma_1"), "r")
Ti			= h5_read(if_plasma,"triangles/temperature")*RefPar.T0eV
if_plasma.close()

Te 	= Te[iWallKnots]
Ti 	= Ti[iWallKnots]

ions = load_ions_list(path)

plot1d_wall_ntmpi
load_eirene_triangles: reading file  c:\Users\Mike.Kryajak\OneDrive\Project\collab\tech\compare_data\soledge_cases\copy_wallStd_bra_nocore_D\triangles.h5
wall_segments
wall_segments: completed
	h5_read: error reading:  /walls_eirene_external/nExtWalls
	h5_read: error reading:  surfaces/nSurfaces
load_eirene_triangles: Completed
get_wall_triangle
get_wall_triangles
get_wall_triangles.2: Completed
get_wall_triangle: Completed


In [22]:
base_plasma_name

'c:\\Users\\Mike.Kryajak\\OneDrive\\Project\\collab\\tech\\compare_data\\soledge_cases\\copy_wallStd_bra_nocore_D\\Results/'

In [23]:
if_plasma

<Closed HDF5 file>

In [44]:
iPlasma = 1
if_plasma	= h5py.File(os.path.join(path, f"Results/plasma_{iPlasma}"), "r")

In [61]:
if_plasma["tempus"][:]

array([0.03548534])

In [43]:
ions

['e-', 'D1+']

In [41]:
for iPlasma in range(len(ions)):
    try:
        if_plasma	= h5py.File(os.path.join(path, f"Results/plasma_{iPlasma}"), "r")
    except:
        print(f"Cannot read plasma file for {ions[iPlasma]}")
        break

    temperature	= h5_read(if_plasma,"triangles/temperature")*RefPar.T0eV
    density		= h5_read(if_plasma,"triangles/density")*RefPar.n0
    velocity	= h5_read(if_plasma,"triangles/velocity")*RefPar.c0

    temperature = temperature[iWallKnots]
    density 	= density[iWallKnots]
    velocity 	= velocity[iWallKnots]

    Jsat		= np.abs(1.6022e-19*velocity*density)*1e-3											#eletronic charge

    M			= velocity/np.sqrt((Te+Ti)/2)*(np.sqrt(RefPar.T0eV)/RefPar.c0)										#warning m_i = 2 for deuterium
    
    if(iPlasma > 0):
        
        try:
            Sn_tri	= h5_read(if_plasma,"triangles/Sn")*RefPar.n0/RefPar.tau0
            if(Sn_tri.max() - Sn_tri.min() > 0.):
                Sn_tri = set_min_positive(Sn_tri)
                Sn	   = Sn_tri[iWallTriangles]
                Sn	   = 0.5*np.append(np.append(Sn[0]+Sn[-1], Sn[:-1]+Sn[1:]), Sn[0]+Sn[-1])
#					print("Sn.min(), Sn.max()",Sn.min(), Sn.max())
            else: Sn = 0.
            Sn_tri = 0
#				Sn	= -h5_read(if_plasma,"triangles/Sn")*RefPar.n0/RefPar.tau0									#recombination
        except:
            print("plot1d_wall_ntmpi: Sn is not available")
            Sn  = 0.

        try:
            Nn	= h5_read(if_plasma,"triangles/Nn")
            Nm	= h5_read(if_plasma,"triangles/Nm")
            Tn	= h5_read(if_plasma,"triangles/Tn")
            Tm	= h5_read(if_plasma,"triangles/Tm")
            
            Nn = Nn[iWallTriangles]
            Nm = Nm[iWallTriangles]
            Tn = Tn[iWallTriangles]
            Tm = Tm[iWallTriangles]
            
            

            Tn = set_min_positive(Tn)
            Tm = set_min_positive(Tm)
            Nn	= 0.5*np.append(np.append(Nn[0]+Nn[-1], Nn[:-1]+Nn[1:]), Nn[0]+Nn[-1])*RefPar.n0
            Nm	= 0.5*np.append(np.append(Nm[0]+Nm[-1], Nm[:-1]+Nm[1:]), Nm[0]+Nm[-1])*RefPar.n0
            Tn	= 0.5*np.append(np.append(Tn[0]+Tn[-1], Tn[:-1]+Tn[1:]), Tn[0]+Tn[-1])
            Tm	= 0.5*np.append(np.append(Tm[0]+Tm[-1], Tm[:-1]+Tm[1:]), Tm[0]+Tm[-1])

            Pn	= (Nn*Tn+Nm*Tm)*1.6e-19
#				Pn	= 0.5*np.append(np.append(Pn[0]+Pn[-1], Pn[:-1]+Pn[1:]), Pn[0]+Pn[-1])

#				Nn	= 0.; Nm = 0.; Tn = 0.; Tm	= 0.
            if(Pn.max() - Pn.min() < 0.): Pn = 0.
        except:
            print("plot1d_wall_ntmpi: Nn is not available")
            Pn  = 0.
    else:
        Sn	= 0.; Pn	= 0.

    if_plasma.close()

plot1d_wall_ntmpi: Sn is not available
	h5_read: error reading:  triangles/Nn
plot1d_wall_ntmpi: Nn is not available


In [42]:
Tn

NameError: name 'Tn' is not defined

In [25]:
#		Skip Phi
phi = 0.
"""
try:
    if_ef	= h5py.File(str(path)+"/Results/electric_fields", "r")
    phi		= h5_read(if_ef,"/triangles/phi")
    if_ef.close()
    phi = phi[iWallKnots]
except:
    phi		= 0.
"""
Fig = []
Ax  = []

yLabels     = ["$n\/(*10^{19}\/m^{-3})$",	  "$T\/(eV)$",	 "$J_{sat}\/(kAm^{-2})$",	"$Mach\/number$"]
yLogLabels  = ["$Ln(n)\/(*10^{19}\/m^{-3})$",  "$Ln(T)\/(eV)$", "$Ln(J_{sat})\/(kAm^{-2})$", "$Mach\/number$"]

if(isinstance(Sn, np.ndarray)):
    yLabels.append("$Ioniz.\/("+ions[iPlasma][:-2]+")$")
    yLogLabels.append("$Ln(Ioniz.)$")

if(isinstance(Pn, np.ndarray)):
    yLabels.append("$n_A\ &\ n_M\ (*10^{19}\ m^{-3})$")
    yLogLabels.append("$Ln(n)$")
    yLabels.append("$T_A\ &\ T_M\ (K)$")
    yLogLabels.append("$Ln(T)$")
    yLabels.append("$P_A+P_m\ (Pa)$")
    yLogLabels.append("$Ln(P)$")
if(isinstance(phi, np.ndarray)):
    yLabels.append("Phi")
    yLogLabels.append("Ln(Phi)")

if(one_plot != 1):
    Fig.append(pyp.figure())
    for i in range(4): Ax.append(Fig[0].add_subplot(2,2,i+1))
    Fig[-1].tight_layout()

    if(len(yLabels) > 4):
        Fig.append(pyp.figure())
        for i in range(len(yLabels)-4): Ax.append(Fig[1].add_subplot(2,2,i+1))
        Fig[-1].tight_layout()
else:
    for i in range(len(yLabels)):
        Fig.append(pyp.figure())
        Ax.append(Fig[i].add_subplot(111))

for figure in Fig:  figure.patch.set_facecolor('white')

Ax[0].set_title(os.path.basename(os.path.abspath(path))+" @ t={:.3f} s".format(RefPar.time))
Ax[1].set_title(ions[iPlasma])

for i in range(len(yLabels)):
    Ax[i].autoscale(enable=True, axis='both', tight=True)
    Ax[i].set_xlabel("$l\ (m)$")
    if(log_scale == 0):
        Ax[i].set_ylabel(yLabels[i])
        Ax[i].set_yscale('linear')
    else:
        Ax[i].set_ylabel(yLogLabels[i])
        Ax[i].set_yscale('log')

#			Ax[i].set_axis_bgcolor("#bdb76b")

In [26]:
#	Plot parameters

pl = []

pl.append(Ax[len(pl)].plot(DistKnots,  density*1e-19,  'b-'))

pl.append(Ax[len(pl)].plot(DistKnots,  temperature,  'b-'))	

pl.append(Ax[len(pl)].plot(DistKnots,  Jsat,  'b-'))

pl.append(Ax[len(pl)].plot(DistKnots, M,  'b-'))
if(exp_data_ok and (iPlasma == 0)):
    Ax[0].plot(exp_data.lp.pos_l,  exp_data.lp.dens*1e-19, 'ro')
    Ax[1].plot(exp_data.lp.pos_l,  exp_data.lp.te,   'ro')
    Ax[2].plot(exp_data.lp.pos_l,  exp_data.lp.jsat*1e-3, 'ro')

if(save == "csv"):
    if(iPlasma == 0): 
        file_header = "l (m)" 
        file_data	= [DistKnots]

    file_data.extend([density*1e-19, temperature, Jsat*1e-3, M])
    if(iPlasma == 0):
        file_header = file_header + ", ne (*10^19 m-3), Te (eV), Jsat_e (kA/m^2), Mach_e"
    else:
        file_header = file_header + ", ni (*10^19 m-3), Ti (eV), Jsat_i (kA/m^2), Mach_i"

if(isinstance(Sn, np.ndarray)):
    Sn = np.where(Sn > 0.1,Sn, 0.1)
    pl.append(Ax[len(pl)].plot(DistKnots,  Sn,  'b-'))
    if(save == "csv"):
        file_data.append(Sn)
        file_header = file_header + ', Ioniz_'+ions[iPlasma][:-2]

if(isinstance(Pn, np.ndarray)):
    Pn = np.where(Pn > 0.1,Pn, 0.1)
    pl.append(Ax[len(pl)].plot(DistKnots,  Nn*1e-19,  'b-'))
    Ax[len(pl)-1].plot(DistKnots,  Nm*1e-19,  'g-')
    pl.append(Ax[len(pl)].plot(DistKnots,  Tn*1.1604e4,  'b-'))
    Ax[len(pl)-1].plot(DistKnots,  Tm*1.1604e4,  'g-')
    pl.append(Ax[len(pl)].plot(DistKnots,  Pn,  'b-'))
    if(save == "csv"):
        file_data.extend([Nn, Tn, Pn])
        file_header = file_header + ', n_'+ions[iPlasma][:-2]+ ' (*10^19 m-3), "T_'+ions[iPlasma][:-2] + ' (eV), P_'+ions[iPlasma][:-2]+" (Pa)"

if(isinstance(phi, np.ndarray)):
    pl.append(Ax[len(pl)].plot(DistKnots,  phi,  'b-'))
    if(save == "csv"):
        file_data.append(Phi)
        file_header = file_header + ', Phi_'+ions[iPlasma][:-2]

if(save != "none"):
    if(save == "csv"):
        np.savetxt("wall_ntmpi_{:s}_t={:.3f}.csv".format(os.path.basename(os.path.abspath(path)),RefPar.time), np.array(file_data).T, delimiter=", ", fmt="%.4e", header= file_header, comments='')
        
    else:
        for i in range(len(Fig)):
            i_plot_file += 1
            if(one_plot != 1): Fig[i].set_size_inches(20.,15.)
            if(save == "pdf"):
                pdf.savefig(Fig[i])
            else:
                Fig[i].savefig("plot1d_wall_ntmpi_t={:.3f}_{:d}.".format(RefPar.time,i_plot_file)+save)

    pyp.show(block=False)
    pyp.close()
else:
    pyp.show()

if(save == "pdf"):	pdf.close()

print("plot1d_wall_ntmpi: Completed")

plot1d_wall_ntmpi: Completed
